# Octave Transposition
---
"In music transposition refers to the process, or operation, of moving a collection of notes (pitches or pitch classes) up or down in pitch by a constant interval." (<a href='https://en.wikipedia.org/wiki/Transposition_(music)'>Wikipedia - Transposition (music)</a>)

_

*Octave Equivalency*

"[...] notes an octave apart are given the same note name in the Western system of music notation — the name of a note an octave above A is also A. This is called octave equivalency, the assumption that pitches one or more octaves apart are musically equivalent in many ways [...]" (<a href='https://en.wikipedia.org/wiki/Octave#Theory'>Wikipedia - Octave, Theory</a>)

_

*Transpositional Equivalence*

"Using integer notation and modulo 12, to transpose a pitch x by n semitones (pitch class transposition by a pitch class interval):" (<a href='https://en.wikipedia.org/wiki/Transposition_(music)#Transpositional_equivalence'>Wikipedia - Transpositional Equivalence</a>)

$T_n(x) = x + n (mod 12)$

_

"Transposing a melody up or down by one octave will not change the key." (<a href='http://brebru.com/musicroom/theory/lesson18/octavetrans.html'>brebru - octave transposition</a>)

In [1]:
import numpy as np

semitones_to_transpose = 12

def transpose_spectrogram (img_arr):
    # check if min/max value out of 12-TET bounds after transposition
    min_oob = False
    max_oob = False
    min_oob_2 = False
    max_oob_2 = False
    if (np.argmax (img_arr[::1,:]>0, axis=0).min ()-semitones_to_transpose < 0):
        print ('  [i] max pitch would be out of bounds after transposition',\
               np.argmax (img_arr[::1,:]>0, axis=0).min ())
        max_oob = True
    if (np.argmax (img_arr[::1,:]>0, axis=0).min ()-semitones_to_transpose*2 < 0):
        print ('  [i] max pitch would be out of bounds after 2x transposition',\
               np.argmax (img_arr[::1,:]>0, axis=0).min ())
        max_oob_2 = True
    if (np.argmax (img_arr[::-1,:]>0, axis=0).min ()-semitones_to_transpose < 0):
        print ('  [i] min pitch would be out of bounds after transposition',\
               np.argmax (img_arr[::-1,:]>0, axis=0).min ())
        min_oob = True
    if (np.argmax (img_arr[::-1,:]>0, axis=0).min ()-semitones_to_transpose*2 < 0):
        print ('  [i] min pitch would be out of bounds after 2x transposition',\
               np.argmax (img_arr[::-1,:]>0, axis=0).min ())
        min_oob_2 = True
    
    
    # go through every column of the image and move every value 12 indices up (or down)
    # - one octave up
    img_arr_trnsp = np.zeros ((4, img_arr.shape[0], img_arr.shape[1])).astype (np.uint8)
    if not max_oob:
        # array with 12 more rows
        img_arr_trnsp[0] = np.vstack (
            (img_arr[semitones_to_transpose:], np.zeros ((semitones_to_transpose, img_arr.shape[1]))))

    # - one octave down
    if not min_oob:
        img_arr_trnsp[1] = np.vstack (
            (np.zeros ((semitones_to_transpose, img_arr.shape[1])), img_arr[:-semitones_to_transpose]))
    
    # - two octaves up
    if not max_oob_2:
        # array with 24 more rows
        img_arr_trnsp[2] = np.vstack (
            (img_arr[semitones_to_transpose*2:], np.zeros ((semitones_to_transpose*2, img_arr.shape[1]))))
    
    # - two octaves down
    if not min_oob_2:
        img_arr_trnsp[3] = np.vstack (
            (np.zeros ((semitones_to_transpose*2, img_arr.shape[1])), img_arr[:-semitones_to_transpose*2]))
    
    
    return img_arr_trnsp

In [2]:
# LOAD SPECTROGRAM FILENAMES
import os
import numpy as np
from sklearn import datasets

PARAM_RND_STATE = 42

container_path = os.path.join ('..', '..', 'src_spectro')
load_content = False

src_spectro_data = datasets.load_files (container_path=container_path,
                                        load_content=load_content,
                                        random_state=PARAM_RND_STATE)
src_spectro_data.keys ()

dict_keys(['target', 'filenames', 'DESCR', 'target_names'])

In [3]:
import os
import numpy as np
from PIL import Image

save_path = os.path.join ('..', '..', 'src_spectro')

for km, spectro_file in enumerate (src_spectro_data.filenames):
    img_arr = np.array (Image.open (spectro_file).convert (mode='L'))
    img_arr_trnsp = transpose_spectrogram (img_arr)

    cnt = 0
    for i in range (img_arr_trnsp.shape[0]):
        if (img_arr_trnsp[i].sum () > 0): # if there's a transposed spectrogram
            # ----- edit images: convert to 8 Bit and rotate by 90 degrees
            base_name, _ = os.path.splitext (os.path.basename (spectro_file))
            target_name = src_spectro_data['target_names'][src_spectro_data['target'][km]]
            save_as = os.path.join (save_path, target_name, base_name + '_{}-oct'.format (i) + '.png')
            print ('>>> saving image as {} ...'.format (save_as), end=' ', flush=True)
            img = Image.fromarray (img_arr_trnsp[i])
            img = img.convert ('RGB')
            img.save (save_as)
            print ('done')
            
            cnt +=1
        
        if (cnt > 1):
            break

>>> saving image as ../../src_spectro/11-0/TRHVOIL128F92E92E6_0-oct.png ... done
>>> saving image as ../../src_spectro/11-0/TRHVOIL128F92E92E6_1-oct.png ... done
>>> saving image as ../../src_spectro/8-0/TRYULKJ128F933B1D9_0-oct.png ... done
>>> saving image as ../../src_spectro/8-0/TRYULKJ128F933B1D9_1-oct.png ... done
>>> saving image as ../../src_spectro/1-1/TRCORZL128F4281888_0-oct.png ... done
>>> saving image as ../../src_spectro/1-1/TRCORZL128F4281888_1-oct.png ... done
>>> saving image as ../../src_spectro/4-1/TRAGVBF12903CC4A59_0-oct.png ... done
>>> saving image as ../../src_spectro/4-1/TRAGVBF12903CC4A59_1-oct.png ... done
>>> saving image as ../../src_spectro/3-0/TRYNAXP128F93205A9_0-oct.png ... done
>>> saving image as ../../src_spectro/3-0/TRYNAXP128F93205A9_1-oct.png ... done
>>> saving image as ../../src_spectro/2-1/TRRJDZB128F14A8C5C_0-oct.png ... done
>>> saving image as ../../src_spectro/2-1/TRRJDZB128F14A8C5C_1-oct.png ... done
>>> saving image as ../../src_spectro/

>>> saving image as ../../src_spectro/0-1/TRHQFBA128F931136E_3-oct.png ... done
>>> saving image as ../../src_spectro/0-1/TRMZWTU128F145E524_0-oct.png ... done
>>> saving image as ../../src_spectro/0-1/TRMZWTU128F145E524_1-oct.png ... done
>>> saving image as ../../src_spectro/6-1/TRSZMTS128F4264B4A_0-oct.png ... done
>>> saving image as ../../src_spectro/6-1/TRSZMTS128F4264B4A_1-oct.png ... done
>>> saving image as ../../src_spectro/2-1/TRKBVJU128F92FCAE0_0-oct.png ... done
>>> saving image as ../../src_spectro/2-1/TRKBVJU128F92FCAE0_1-oct.png ... done
>>> saving image as ../../src_spectro/5-0/TRTPAWR12903D017C7_0-oct.png ... done
>>> saving image as ../../src_spectro/5-0/TRTPAWR12903D017C7_1-oct.png ... done
>>> saving image as ../../src_spectro/10-1/TRZNUPL128F934D3B0_0-oct.png ... done
>>> saving image as ../../src_spectro/10-1/TRZNUPL128F934D3B0_1-oct.png ... done
>>> saving image as ../../src_spectro/0-1/TREQDOZ12903CAAD59_0-oct.png ... done
>>> saving image as ../../src_spectro/

>>> saving image as ../../src_spectro/1-1/TRTMPFS128F14698A0_0-oct.png ... done
>>> saving image as ../../src_spectro/1-1/TRTMPFS128F14698A0_1-oct.png ... done
  [i] max pitch would be out of bounds after 2x transposition 21
>>> saving image as ../../src_spectro/4-1/TRNKPYS128F428AA73_0-oct.png ... done
>>> saving image as ../../src_spectro/4-1/TRNKPYS128F428AA73_1-oct.png ... done
>>> saving image as ../../src_spectro/7-0/TRCMPVE128F9332C70_0-oct.png ... done
>>> saving image as ../../src_spectro/7-0/TRCMPVE128F9332C70_1-oct.png ... done
  [i] min pitch would be out of bounds after 2x transposition 23
>>> saving image as ../../src_spectro/1-0/TRSUMDT128F1493B47_0-oct.png ... done
>>> saving image as ../../src_spectro/1-0/TRSUMDT128F1493B47_1-oct.png ... done
>>> saving image as ../../src_spectro/4-1/TRDMROQ128F14A597A_0-oct.png ... done
>>> saving image as ../../src_spectro/4-1/TRDMROQ128F14A597A_1-oct.png ... done
>>> saving image as ../../src_spectro/4-0/TROEPPK128F92F33EC_0-oct.png

>>> saving image as ../../src_spectro/10-0/TRQEAPQ128F92F9A58_1-oct.png ... done
>>> saving image as ../../src_spectro/10-0/TRQEAPQ128F92F9A58_3-oct.png ... done
>>> saving image as ../../src_spectro/4-1/TRIGSHJ128F92CA48D_0-oct.png ... done
>>> saving image as ../../src_spectro/4-1/TRIGSHJ128F92CA48D_1-oct.png ... done
  [i] min pitch would be out of bounds after 2x transposition 23
>>> saving image as ../../src_spectro/11-0/TRIEVUW128F4239521_0-oct.png ... done
>>> saving image as ../../src_spectro/11-0/TRIEVUW128F4239521_1-oct.png ... done
>>> saving image as ../../src_spectro/5-0/TRTMIYO128F1473297_0-oct.png ... done
>>> saving image as ../../src_spectro/5-0/TRTMIYO128F1473297_1-oct.png ... done
>>> saving image as ../../src_spectro/0-0/TRYGKEI12903D08088_0-oct.png ... done
>>> saving image as ../../src_spectro/0-0/TRYGKEI12903D08088_1-oct.png ... done
>>> saving image as ../../src_spectro/1-0/TRLZZOJ128F1494C12_0-oct.png ... done
>>> saving image as ../../src_spectro/1-0/TRLZZOJ12

>>> saving image as ../../src_spectro/5-0/TRHDYVW128F421A79D_1-oct.png ... done
  [i] max pitch would be out of bounds after 2x transposition 21
>>> saving image as ../../src_spectro/7-0/TRROBXQ128F428D2BD_0-oct.png ... done
>>> saving image as ../../src_spectro/7-0/TRROBXQ128F428D2BD_1-oct.png ... done
  [i] max pitch would be out of bounds after 2x transposition 16
>>> saving image as ../../src_spectro/3-0/TRNNXEX128F42B72D5_0-oct.png ... done
>>> saving image as ../../src_spectro/3-0/TRNNXEX128F42B72D5_1-oct.png ... done
>>> saving image as ../../src_spectro/10-0/TRGYVLB128F4295F6B_0-oct.png ... done
>>> saving image as ../../src_spectro/10-0/TRGYVLB128F4295F6B_1-oct.png ... done
>>> saving image as ../../src_spectro/3-0/TRHWFHO128F93130DA_0-oct.png ... done
>>> saving image as ../../src_spectro/3-0/TRHWFHO128F93130DA_1-oct.png ... done
>>> saving image as ../../src_spectro/2-1/TRYPLQR128F933ADCA_0-oct.png ... done
>>> saving image as ../../src_spectro/2-1/TRYPLQR128F933ADCA_1-oct.p

>>> saving image as ../../src_spectro/1-1/TRFGVOO128F92F9E2D_0-oct.png ... done
>>> saving image as ../../src_spectro/1-1/TRFGVOO128F92F9E2D_1-oct.png ... done
>>> saving image as ../../src_spectro/4-0/TRGYFTS128F9306691_0-oct.png ... done
>>> saving image as ../../src_spectro/4-0/TRGYFTS128F9306691_1-oct.png ... done
>>> saving image as ../../src_spectro/11-0/TRDHTMJ128F429FAED_0-oct.png ... done
>>> saving image as ../../src_spectro/11-0/TRDHTMJ128F429FAED_1-oct.png ... done
>>> saving image as ../../src_spectro/5-1/TRKKPSE128F933ECEF_0-oct.png ... done
>>> saving image as ../../src_spectro/5-1/TRKKPSE128F933ECEF_1-oct.png ... done
>>> saving image as ../../src_spectro/1-0/TRKREFV128F425C19D_0-oct.png ... done
>>> saving image as ../../src_spectro/1-0/TRKREFV128F425C19D_1-oct.png ... done
>>> saving image as ../../src_spectro/8-1/TRNUNEZ128F9319C24_0-oct.png ... done
>>> saving image as ../../src_spectro/8-1/TRNUNEZ128F9319C24_1-oct.png ... done
>>> saving image as ../../src_spectro/

>>> saving image as ../../src_spectro/9-1/TRZURID128F933F6C3_0-oct.png ... done
>>> saving image as ../../src_spectro/9-1/TRZURID128F933F6C3_1-oct.png ... done
>>> saving image as ../../src_spectro/0-1/TRGZFIS128F92FF25A_0-oct.png ... done
>>> saving image as ../../src_spectro/0-1/TRGZFIS128F92FF25A_1-oct.png ... done
>>> saving image as ../../src_spectro/4-1/TRQLDMB128F421D3D6_0-oct.png ... done
>>> saving image as ../../src_spectro/4-1/TRQLDMB128F421D3D6_1-oct.png ... done
>>> saving image as ../../src_spectro/10-1/TRPVEPJ128F92FB977_0-oct.png ... done
>>> saving image as ../../src_spectro/10-1/TRPVEPJ128F92FB977_1-oct.png ... done
  [i] max pitch would be out of bounds after 2x transposition 20
>>> saving image as ../../src_spectro/2-0/TRINSOP128F14A9167_0-oct.png ... done
>>> saving image as ../../src_spectro/2-0/TRINSOP128F14A9167_1-oct.png ... done
>>> saving image as ../../src_spectro/9-0/TRAKXUV128F9301F49_0-oct.png ... done
>>> saving image as ../../src_spectro/9-0/TRAKXUV128F

---

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

#spectro_file = os.path.join ('..', 'fft', 'test', 'c-major-scale-on-treble-clef.png')
#spectro_file = os.path.join ('..', '..', 'src_spectro', '8-0', 'TRIZVHE128EF351C4E.png')
#spectro_file = os.path.join ('..', '..', 'src_spectro', '7-0', 'TRUIIZI12903D00069.png')
spectro_file = os.path.join ('..', '..', 'src_spectro', '7-1', 'TRNIWUC128F4284F76.png')

img = Image.open (spectro_file).convert (mode='L')

plt.imshow (img)
plt.show ()

In [ ]:
img.size

In [ ]:
img_arr = np.array (img)
img_arr.shape

In [ ]:
k = img_arr.shape[0]-np.argmax (img_arr, axis=0)
print (k, k.shape)

In [ ]:
# run through all time chunks, get the corresponding pitch and add it to a list if not already in list

list_of_pitches_midi = np.empty (1).astype (int)
max_pitch_midi = img_arr.shape[0]
for i in range (img_arr.shape[1]):
    
    # check first if we have absolute silence in this time chunk = all values are zero
    if (img_arr[:,i].sum () == 0):
        continue
    
    pitch_midi = max_pitch_midi - img_arr[:,i].argmax ()
    if pitch_midi not in list_of_pitches_midi:
        list_of_pitches_midi = np.append (list_of_pitches_midi, pitch_midi)

list_of_pitches_midi = list_of_pitches_midi[1:]
print (list_of_pitches_midi)

In [ ]:
# run through all time chunks, get the corresponding pitch and add it to a list if not already in list

#list_of_pitches_midi = np.empty (1).astype (int)
max_pitch_midi = img_arr.shape[0]
#for i in range (img_arr.shape[1]):
    
    # check first if we have absolute silence in this time chunk = all values are zero
    #if (img_arr[:,i].sum () == 0):
    #    continue
    
list_of_pitches_midi_up = np.argmax (img_arr[::1,:] > 0, axis=0)
list_of_pitches_midi_dwn = np.argmax (img_arr[::-1,:] > 0, axis=0)
    #if pitch_midi not in list_of_pitches_midi:
    #    list_of_pitches_midi = np.append (list_of_pitches_midi, pitch_midi)

#list_of_pitches_midi = list_of_pitches_midi[1:]
print (list_of_pitches_midi_up, list_of_pitches_midi_up.size)
print (list_of_pitches_midi_dwn, list_of_pitches_midi_dwn.size)

In [ ]:
#plt.imshow (img_arr[::1,:])
plt.imshow (img_arr[::-1,:])

In [ ]:
# function: get key of array pixel value
import math

def get_music_key (arr_val):
    tone = arr_val % 12
    octave = math.floor (arr_val / 12) - 1
    
    return tone, octave

In [ ]:
dict_music_keys = {'C':0, 'C#':1, 'D':2, 'D#':3, 'E':4, 'F':5, 'F#':6, 'G':7, 'G#':8, 'A':9, 'A#':10, 'B':11}

In [ ]:
for arr_val in list_of_pitches_midi:
    for dict_key, dict_value in dict_music_keys.items ():
        tone, octave = get_music_key (arr_val)
        if (tone == dict_value):
            print ('{}{}'.format (dict_key, octave), end=', ', flush=True)
print ()

In [ ]:
img_arr_trnsp = transpose_spectrogram (img_arr)

cnt = 0
for i in range (img_arr_trnsp.shape[0]):
    if (img_arr_trnsp[i].sum () > 0):
        cnt +=1

In [ ]:
fig, axs= plt.subplots (1, cnt+1, sharex=True, sharey=True)
axs[0].imshow (Image.fromarray (img_arr))
k = 1
for i in range (img_arr_trnsp.shape[0]):
    if (img_arr_trnsp[i].sum () > 0):
        axs[k].imshow (Image.fromarray (img_arr_trnsp[i]))
        k += 1

fig.set_size_inches (16, 14)
plt.show ()

In [ ]:
list_of_pitches_midi_oct_up = np.empty (1).astype (int)
max_pitch_midi_oct_up = img_arr_oct_up.shape[0]
for i in range (img_arr_oct_up.shape[1]):
    
    # check first if we have absolute silence in this time chunk = all values are zero
    if (img_arr_oct_up[:,i].sum () == 0):
        continue
    
    pitch_midi = max_pitch_midi_oct_up - img_arr_oct_up[:,i].argmax ()
    if pitch_midi not in list_of_pitches_midi_oct_up:
        list_of_pitches_midi_oct_up = np.append (list_of_pitches_midi_oct_up, pitch_midi)

list_of_pitches_midi_oct_up = list_of_pitches_midi_oct_up[1:]
print (list_of_pitches_midi_oct_up)



list_of_pitches_midi_oct_dwn = np.empty (1).astype (int)
max_pitch_midi_oct_dwn = img_arr_oct_dwn.shape[0]
for i in range (img_arr_oct_dwn.shape[1]):
    
    # check first if we have absolute silence in this time chunk = all values are zero
    if (img_arr_oct_dwn[:,i].sum () == 0):
        continue
    
    pitch_midi = max_pitch_midi_oct_dwn - img_arr_oct_dwn[:,i].argmax ()
    if pitch_midi not in list_of_pitches_midi_oct_dwn:
        list_of_pitches_midi_oct_dwn = np.append (list_of_pitches_midi_oct_dwn, pitch_midi)

list_of_pitches_midi_oct_dwn = list_of_pitches_midi_oct_dwn[1:]
print (list_of_pitches_midi_oct_dwn)

In [ ]:
for arr_val in list_of_pitches_midi_oct_up:
    for dict_key, dict_value in dict_music_keys.items ():
        tone, octave = get_music_key (arr_val)
        if (tone == dict_value):
            print ('{}{}'.format (dict_key, octave), end=', ', flush=True)
print ()

for arr_val in list_of_pitches_midi_oct_dwn:
    for dict_key, dict_value in dict_music_keys.items ():
        tone, octave = get_music_key (arr_val)
        if (tone == dict_value):
            print ('{}{}'.format (dict_key, octave), end=', ', flush=True)
print ()

print ()
for arr_val in list_of_pitches_midi:
    for dict_key, dict_value in dict_music_keys.items ():
        tone, octave = get_music_key (arr_val)
        if (tone == dict_value):
            print ('{}{}'.format (dict_key, octave), end=', ', flush=True)
print ()